In [44]:
import re
labels = {}
notes = {}
refs = {}

def additem(curitem,curdict):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1
def loeb2(voln):
    global labels
    global notes
    global refs
    fname = 'work1-lucian-loeb0' +str(voln)+ '.1st1k-eng1.xml'
    print('\n'+fname)
    f = open(fname)
    outfname = re.sub('work1','work2',fname)
    outf = open(outfname,'w')
    prevnote = 0
    curnote = 0
    prevref = 0
    curpage = ''
    cursection = ''
    closenote = 0
    prevsect = 0
    inbody = 0
    lastchar = ''
    
    for l in f:
        l = re.sub('\s+$','',l)
        if(voln == 1):
            print(l,file=outf)
            continue
        if(re.search('<body',l)):
            inbody = 1
        if(not inbody):
            print(l,file=outf)
            continue
        if(not prevnote and not re.search('<note',l)):
            l = re.sub('(”|[a-z\?])\s+([0-9])','\g<1>\g<2>',l)
        
        m = re.search('<pb n="([^"]+)"',l)
        if(m):
            if(closenote):
                print('</p></note>\n\n',file=outf)
            curpage = m[1]
            closenote = 0
            prevref = 0
            prevnote = 0
        
        m = re.search('^([A-Z\-]+)[^a-zA-Z]+$',l)
        if(m):
            print(l)
        m = re.search('^([A-Z\-]+)$',l)
        if(m):
            additem(m[1],labels)
        l = re.sub('^([A-Z]+)$','<label>\g<1></label>',l)
        
        refpat = '([\\.\’;:,\!\?”a-z—])([1-9])'
        
        
        m = re.search('^([1-9])\s+',l)
        if(m):
            closenote = 1
            if(m[1] == '1'):
                l = re.sub('^([1-9])\s+','<note n="'+curpage+'.\g<1>"><p>',l)
            else:
                l = re.sub('^([1-9])\s+','</p></note>\n\n<note n="'+curpage+'.\g<1>"><p>',l)
            curnote = int(m[1])
            if(curnote != prevnote + 1):
                print('notejump',voln,curpage,prevnote,curnote)
                
        l = re.sub('<note n="([0-9]+)"','<note n="'+curpage+'.\g<1>"',l)
                
        m = re.search('<note n="[0-9]+\.([0-9]+)"',l)
        if(m):
            additem(str(voln)+'.'+curpage+'.'+m[1],notes)
            prevnote = int(m[1])
        
        if(re.search('type="(translation|edition)"',l)):
            cursection = ''
            print(curpage,l)
            
        m = re.search('type="section" n="([0-9]+)"',l)
        if(m):
            cursection = m[1]
                           
        if(cursection and not prevnote and curpage and not re.search('(<bibl|<note)',l)):
            while(re.search(refpat,l)):
                m = re.search(refpat,l)
                #print(curpage,m[2])
                curref = int(m[2])
                additem(str(voln)+'.'+curpage+'.'+m[2],refs)
                l = re.sub(refpat,'\g<1><ref n="'+curpage+'-\g<2>"/>',l,1)
                if(curref != prevref + 1):
                    print('refjump',voln,curpage,prevnote,prevref,curref,l)
                    
                prevref = curref
            #print(l)
                           

        l = re.sub('(<ref n="[0-9]+)-','\g<1>.',l)
        m = re.search('<ref n="([0-9]+)"',l)
        if(m):
            additem(str(voln)+'.'+curpage+'.'+m[1],refs)

            l = re.sub('<ref n="([0-9]+)"','<ref n="'+curpage+'.\g<1>"',l)
        
        m = re.search('section" n="([0-9]+)"',l)
        if(m):
            cursect = int(m[1])
            prevsect = cursect
        m = re.search('@([0-9]+)',l)
        if(m):
            cursect = int(m[1])
            if(cursect != prevsect + 1):
                print('skips',curpage,prevsect,cursect)
            l = re.sub('@([0-9]+)','</p></div>\n\n<div type="textpart" subtype="section" n="\g<1>"><p>',l)
            prevsect= cursect
            
        if(re.search('@',l)):
            cursect = prevsect + 1
            l = re.sub('@','</p></div>\n\n<div type="textpart" subtype="section" n="'+str(cursect)+'"><p>',l)
            prevsect = cursect
        
        if( lastchar != '>' and re.search('<label>',l)):
           l = re.sub('<label>','</p>\n<p><label>',l)
        m = re.search('(.)$',l)
        if(m):
            lastchar  = m[1]
        print(l,file=outf)
    f.close()
    outf.close()
    for foo in refs:
        if(not foo in notes):
            print('no note for',foo)

    for foo in notes:
        if(not foo in refs):
            print('no ref for',foo)
 
for i in range(5,6):
    loeb2(i)

for foo in sorted(labels,key=labels.get,reverse=True):
    print(foo,labels[foo])


work1-lucian-loeb05.1st1k-eng1.xml
3 <div type="translation" n="tlg0062.tlg042">
55 <div type="translation" n="tlg0062.tlg043">
103 <div type="translation" n="tlg0062.tlg044">
211 <div type="translation" n="tlg0062.tlg045">
293 <div type="translation" n="tlg0062.tlg046">
331 <div type="translation" n="tlg0062.tlg047">
349 <div type="translation" n="tlg0062.tlg048">
373 <div type="translation" n="tlg0062.tlg049">
419 <div type="translation" n="tlg0062.tlg050">
445 <div type="translation" n="tlg0062.tlg051">
477 <div type="translation" n="tlg0062.tlg052">
LYCINUS 26
ZEUS 22
HERMES 21
MNESIPPUS 21
TOXARIS 20
PHILOSOPHY 14
LEXIPHANES 10
PAMPHILUS 9
MOMUS 8
HERACLES 7
HUSBAND 6
CRATO 6
APOLLO 5
WOMAN 4
ORPHEUS 3
SOPOLIS 3
SCARABEE 1
I 1


In [11]:
len(refs)

164

In [10]:
import re

scansources = {
    
    1: 'https://hdl.handle.net/2027/uc1.b000604153',
    2: 'https://hdl.handle.net/2027/uc1.b000604154',
    3: 'https://hdl.handle.net/2027/osu.32435024202137',
    4: 'https://hdl.handle.net/2027/osu.32435024202129',
    5: 'https://hdl.handle.net/2027/osu.32435024202111'
}

dates = {
    
    1:'1913',
    2:'1915',
    3:'1921',
    4:'1925',
    5:'1936'
}

def cleanl(s):
    s = re.sub('\s+$','',s)
    s = re.sub('<note type="head">[^<]+</head>','',s)
    return(s)

def doloebeng(n):
    pagenum = 0
    prevsect = 0
    curpage = ''
    outfname = "work-lucian-loeb0" + str(n) +'.1st1K-eng1.xml'
    outf = open(outfname,'w')
    f = open('txt/teiheader')
    for l in f:
        l = re.sub('\s+$','',l)
        l = re.sub('<date>1961','<date>'+dates[n],l)
        l = re.sub('<biblScope unit="volume">1','<biblScope unit="volume">' + str(n),l)
        if(re.search('https://archive.org/details/lucianha01luciuoft',l)):
            l = re.sub('https://archive.org/details/lucianha01luciuoft',scansources[n],l)
        print(l,file=outf)
    
    f.close()

    fname = 'txt/curvol' + str(n) + '-eng.txt'
    f = open(fname)
    for l in f:
        l = cleanl(l)
        m = re.search('<pb n="([0-9]+)',l)
        if(m):
            curpage = m[1]
        if(not curpage):
            continue
        if(re.search('[02468]$',curpage)):
            continue
        m = re.search('section" n="([0-9]+)"',l)
        if(m):
            cursect = int(m[1])
            prevsect = cursect
        m = re.search('@([0-9]+)',l)
        if(m):
            cursect = int(m[1])
            if(cursect != prevsect + 1):
                print('skips',curpage,prevsect,cursect)
            l = re.sub('@([0-9]+)','</p></div>\n\n<div type="textpart" subtype="section" n="\g<1>"><p>',l)
            prevsect= cursect
            
        if(re.search('@',l)):
            cursect = prevsect + 1
            l = re.sub('@','</p></div>\n\n<div type="textpart" subtype="section" n="'+str(cursect)+'"><p>',l)
            prevsect = cursect
        print(l,file=outf)
            
    
    outf.close()
    
    f.close()
    
doloebeng(3)